## Group Project - Laptop Sales

In [1]:
# Install folium using conda install -c conda-forge folium

In [2]:
import pandas as pd
import folium
import os

In [3]:
# Read xlsx data into a pandas dataframe. 

# NOTE: The xlsx file must be closed in order for pandas to load the data.
fn = 'LaptopSalesLL.xlsx'
myDF = pd.read_excel(fn)

In [4]:
myDF.head()

,Date,Configuration,Customer Postcode,Store Postcode,Retail Price,Screen Size (Inches),Battery Life (Hours),RAM (GB),Processor Speeds (GHz),Integrated Wireless?,HD Size (GB),Bundled Applications?,customer X,customer Y,store X,store Y,Customer Latitude,Customer Longitude,Store Latitude,Store Longitude
0,2008-01-01 00:01:19,163,EC4V 5BH,SE1 2BN,455.0,15,5,1,2.0,Yes,80,Yes,532041,180995,534057.0,179682.0,51.512579,-0.098731,51.500186,-0.070057
1,2008-01-01 00:02:52,320,SW4 0JL,SW12 9HD,545.0,15,6,1,2.0,No,300,No,529240,175537,528739.0,173080.0,51.464129,-0.140912,51.442082,-0.149046
2,2008-01-01 00:04:18,23,EC3V 1LR,E2 0RY,515.0,15,4,1,2.0,Yes,300,Yes,533095,181047,535652.0,182961.0,51.512699,-0.083435,51.529346,-0.045410
3,2008-01-01 00:04:40,169,SW1P 3AU,SE1 2BN,395.0,15,5,1,2.0,No,40,Yes,529902,179641,534057.0,179682.0,51.500787,-0.129897,51.500186,-0.070057
4,2008-01-01 00:06:04,365,EC4V 4EG,SW1V 4QQ,585.0,15,6,2,2.0,No,120,Yes,531684,180948,528924.0,178440.0,51.51212,-0.103737,51.490255,-0.144474


**Clean the data**

In [5]:
# Remove non-numeric values from customer lat and long columns
myDF = myDF[myDF['Customer Latitude'] != "FAILED"]
myDF = myDF[myDF['Customer Longitude'] != "FAILED"]

In [6]:
sales = myDF[['Customer Latitude','Customer Longitude','Store Latitude','Store Longitude']]

In [7]:
sales.isnull().sum()

Customer Latitude       0
Customer Longitude      0
Store Latitude        147
Store Longitude       147
dtype: int64

In [8]:
# Remove rows containing NA for store latitude and longitude
sales = sales[pd.notnull(sales['Store Latitude'])]

In [9]:
sales.head()

,Customer Latitude,Customer Longitude,Store Latitude,Store Longitude
0,51.512579,-0.098731,51.500186,-0.070057
1,51.464129,-0.140912,51.442082,-0.149046
2,51.512699,-0.083435,51.529346,-0.045410
3,51.500787,-0.129897,51.500186,-0.070057
4,51.51212,-0.103737,51.490255,-0.144474


In [10]:
# Convert lat/lon objects to numeric
sales = sales.apply(pd.to_numeric, errors='ignore')

In [11]:
def store_sales(df):
    # Calculates the number of transactions per store
    count_series = df.groupby(['Store Latitude', 'Store Longitude']).size()
    storesales = count_series.to_frame(name='size').reset_index()
    return storesales

In [12]:
store_sales(sales)

,Store Latitude,Store Longitude,size
0,51.398191,-0.093631,7832
1,51.418947,-0.305625,4333
2,51.442082,-0.149046,12673
3,51.461802,-0.199785,14069
4,51.483307,-0.025775,16584
5,51.485033,-0.279066,6536
6,51.490255,-0.144474,43322
7,51.500186,-0.070057,44491
8,51.500787,-0.129897,61663
9,51.519370,-0.211483,11726


In [13]:
def customer_purchases(df):
    # Calculates the number of purchases per customer
    count_series = df.groupby(['Customer Latitude', 'Customer Longitude']).size()
    cus_purchases = count_series.to_frame(name='Number of Purchases').reset_index()
    locations = df.drop_duplicates(subset=['Customer Latitude', 'Customer Longitude'])   
    customer_purchases =  pd.merge(cus_purchases, locations, how='outer')
    
    return customer_purchases

In [14]:
customer_purchases(sales)

,Customer Latitude,Customer Longitude,Number of Purchases,Store Latitude,Store Longitude
0,51.367827,-0.103900,325,51.398191,-0.093631
1,51.368895,-0.099891,330,51.461802,-0.199785
2,51.371064,-0.100562,354,51.442082,-0.149046
3,51.372037,-0.100091,356,51.442082,-0.149046
4,51.372889,-0.101593,337,51.461802,-0.199785
5,51.378117,-0.088530,388,51.461802,-0.199785
6,51.379161,-0.089134,376,51.461802,-0.199785
7,51.379348,-0.086855,366,51.461802,-0.199785
8,51.379574,-0.081960,351,51.483307,-0.025775
9,51.380219,-0.094464,348,51.461802,-0.199785


In [15]:
def plot_map(df):
    # Generate base map
    folium_map = folium.Map(location=[df['Store Latitude'].mean(), df['Store Longitude'].mean()],
                            zoom_start=10,
                            tiles='cartodbpositron',
                            width='100%',
                            zoom_control=True)
    stores = store_sales(df)
    cus = customer_purchases(df)
    
    for index, row in stores.iterrows():
        # Add marker for each customer
        store_location = (row['Store Latitude'], row['Store Longitude'])
        store_radius = row['size']/1000
        sales =   str(int(row['size']))
        
        # Define Store Layers
        totalSales = store_sales(myDF)
        ss = totalSales.at[index, 'size']
        store_name = 'Store {}'.format(index+1)
        store_popup = 'Store {} <br> Total Transactions: {}'.format(index+1, ss)
        store_layer = folium.FeatureGroup(name = store_name, show=False)
        
        for index, row in cus.iterrows():
            cus_location = (row['Customer Latitude'], row['Customer Longitude'])
            
            if ((row['Store Latitude'] , row['Store Longitude']) == store_location) : 
                numPurchases = customer_purchases(myDF)
                purchases = numPurchases.at[index, 'Number of Purchases']
                cust_popup = 'Customer {} <br> Total Purchases: {}'.format(index+1, purchases)
                
                folium.vector_layers.CircleMarker(location=cus_location,color = "royalblue",fill=True, radius=1, opacity=0.7, 
                                   popup=cust_popup).add_to(folium_map)
                cline = folium.PolyLine(locations=[store_location, cus_location], color='mediumseagreen',weight = 1)
                store_layer.add_child(cline)


        folium_map.add_child(store_layer)   
        
        folium.CircleMarker(location=store_location,
                     radius = store_radius,
                            color ='tomato',fill=True,
                            opacity = 0.6
                     ).add_to(folium_map)

        folium.Marker(location=store_location,
                     icon=folium.Icon(icon='star', color = 'red'),
                     popup=store_popup,
                     tooltip = store_name
                     ).add_to(folium_map)
        
        
    # Add layer control to the map
    folium.LayerControl(collapsed=True).add_to(folium_map)     
    
    return folium_map

### Store and Customer Network Map
To see the specific store-consumer network:
- View the store id and number of transactions by the store star icon
- Select the corresponding store id on the legend to view its customers
- The blue points represent the customers. View their customer id and total number of transactions by clicking.


In [16]:
plot_map(sales)